In [1]:
import os
import pandas as pd

/var/folders/y4/c4phbshn5739xq523fgf_cl40000gn/T/ipykernel_26894/3571106454.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [7]:
def convert_string_to_bool(in_string):
    """if string ="" False, else True"""
    if in_string:
        return True
    else:
        return False

def convert_string_to_bool_int(in_string):
    """if string ="" False, else True"""
    if in_string:
        return 1
    else:
        return 0

def convert_bool_string_to_bool_int(in_string):
    """ if string==False -> False, etc."""
    if in_string=="(False,)" or in_string=="False":
        return 0
    elif in_string=="(True,)" or in_string=="True":
        return 1
    else:
        print(type(in_string))
        print(in_string)
        print(in_string[0])
        raise Exception("Error in Transforming BoolString to Int")

def modify_text_prompt(keys_removed):
    """changes to short version"""
    keys = keys_removed.count("+")
    if keys==0:
        return keys_removed
    elif keys>1:
        return "short"
    else:
        return "long"

In [34]:
CURRENT_TRIAL_NAME = "v2_eval_20_3"
file_path = get_data_file_path(current_trial_name=CURRENT_TRIAL_NAME)
results_data = pd.read_csv(
    os.path.join("..",file_path), 
    converters = {
        "early_stop":convert_string_to_bool_int, 
        "error": convert_string_to_bool_int,
        "done": convert_bool_string_to_bool_int,
        "success": convert_bool_string_to_bool_int,
        "keys_removed": modify_text_prompt
    }
)

header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error", "env_idx"]
clean_results = results_data[header_order]
clean_results.head()

,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error,env_idx
0,gpt-3.5-turbo-0125,react-1,0.0,cool,0,1,0,49,12,0,0,0
1,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,13,0,0,1
2,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,33,0,0,2
3,gpt-3.5-turbo-0125,react-1,0.0,put,0,1,0,49,26,0,0,3
4,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,39,2,0,4


In [61]:
# This is the table that shows results per env_type
out = clean_results.groupby(["model","keys_removed","env_type"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","env_type","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby(["env_type"])
x.head()

,model,keys_removed,temperature,env_type,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,agentbench-v2,0.0,clean,4,0,0,4,56,30,20
1,gpt-3.5-turbo-0125,agentbench-v2,0.0,cool,5,0,2,3,126,64,15
2,gpt-3.5-turbo-0125,agentbench-v2,0.0,examine,2,1,2,0,64,26,1
3,gpt-3.5-turbo-0125,agentbench-v2,0.0,heat,3,0,2,1,115,65,9
4,gpt-3.5-turbo-0125,agentbench-v2,0.0,put,6,2,3,3,170,81,32
5,gpt-3.5-turbo-0125,jsonreact-1,0.0,clean,4,0,3,1,183,89,5
6,gpt-3.5-turbo-0125,jsonreact-1,0.0,cool,5,0,1,4,187,109,26
7,gpt-3.5-turbo-0125,jsonreact-1,0.0,examine,2,0,1,1,56,44,7
8,gpt-3.5-turbo-0125,jsonreact-1,0.0,heat,3,1,2,1,75,32,6
9,gpt-3.5-turbo-0125,jsonreact-1,0.0,put,6,0,5,1,274,158,20


In [48]:
out.to_csv("../results/pandas_test_output.csv",sep="&",index=False)

In [62]:
print(out[["model", "keys_removed","env_type","total","success"]])

                 model   keys_removed env_type  total  success
0   gpt-3.5-turbo-0125  agentbench-v2    clean      4        0
1   gpt-3.5-turbo-0125  agentbench-v2     cool      5        0
2   gpt-3.5-turbo-0125  agentbench-v2  examine      2        1
3   gpt-3.5-turbo-0125  agentbench-v2     heat      3        0
4   gpt-3.5-turbo-0125  agentbench-v2      put      6        2
5   gpt-3.5-turbo-0125    jsonreact-1    clean      4        0
6   gpt-3.5-turbo-0125    jsonreact-1     cool      5        0
7   gpt-3.5-turbo-0125    jsonreact-1  examine      2        0
8   gpt-3.5-turbo-0125    jsonreact-1     heat      3        1
9   gpt-3.5-turbo-0125    jsonreact-1      put      6        0
10  gpt-3.5-turbo-0125    jsonreact-2    clean      4        0
11  gpt-3.5-turbo-0125    jsonreact-2     cool      5        1
12  gpt-3.5-turbo-0125    jsonreact-2  examine      2        0
13  gpt-3.5-turbo-0125    jsonreact-2     heat      3        1
14  gpt-3.5-turbo-0125    jsonreact-2      put      6  

In [63]:
out.pivot(index=["keys_removed"],columns=["env_type"], values=["success"])

success                      
env_type        clean cool examine heat put
keys_removed                               
agentbench-v2       0    0       1    0   2
jsonreact-1         0    0       0    1   0
jsonreact-2         0    1       0    1   2
long                2    1       0    2   0
react-1             0    0       1    0   0
react-2             0    1       1    1   2
short               2    1       1    2   3

In [64]:
# This is the table that shows total results.
out = clean_results.groupby(["model","keys_removed"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby("keys_removed")
x.head()

,model,keys_removed,temperature,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,agentbench-v2,0.0,20,3,9,11,531,266,77
1,gpt-3.5-turbo-0125,jsonreact-1,0.0,20,1,12,8,775,432,64
2,gpt-3.5-turbo-0125,jsonreact-2,0.0,20,4,10,10,517,318,99
3,gpt-3.5-turbo-0125,long,0.0,20,5,12,8,594,302,54
4,gpt-3.5-turbo-0125,react-1,0.0,20,1,19,1,915,478,14
5,gpt-3.5-turbo-0125,react-2,0.0,20,5,15,5,705,359,35
6,gpt-3.5-turbo-0125,short,0.0,20,9,15,5,526,261,38


In [65]:
# This is the table that shows results per env_id
out = clean_results.groupby(["model","env_idx","keys_removed"]).sum().reset_index()
out_header_order = ["model", "keys_removed","temperature","env_idx","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby("env_idx")
x.head()

,model,keys_removed,temperature,env_idx,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,agentbench-v2,0.0,0,0,0,1,9,7,5
1,gpt-3.5-turbo-0125,jsonreact-1,0.0,0,0,0,1,33,23,5
2,gpt-3.5-turbo-0125,jsonreact-2,0.0,0,1,1,0,9,0,0
3,gpt-3.5-turbo-0125,long,0.0,0,0,0,1,33,21,7
4,gpt-3.5-turbo-0125,react-1,0.0,0,0,1,0,49,12,0
...,...,...,...,...,...,...,...,...,...,...
133,gpt-3.5-turbo-0125,agentbench-v2,0.0,19,0,1,0,49,31,0
134,gpt-3.5-turbo-0125,jsonreact-1,0.0,19,0,1,0,49,22,0
135,gpt-3.5-turbo-0125,jsonreact-2,0.0,19,0,1,0,49,34,0
136,gpt-3.5-turbo-0125,long,0.0,19,1,1,0,7,0,0


In [67]:
out.pivot(index=["keys_removed"],columns=["env_idx"], values=["success"])

success                                                         
env_idx            0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19
keys_removed                                                                  
agentbench-v2       0  0  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0
jsonreact-1         0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
jsonreact-2         1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  1  0
long                0  0  1  0  0  0  0  0  0  0  0  1  1  0  0  0  0  1  0  1
react-1             0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
react-2             1  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  0  1  0
short               0  1  0  1  0  0  1  1  0  0  0  1  1  1  0  0  0  1  0  1